In [1]:
import cluj_tools as ct

Today is 2020-02-02


In [2]:
n_samples = 1000

### Pull injured players

In [3]:
injured_out = ct.pull_injured_players()

### Get matchup end date and period

In [4]:
matchup_end_date, matchupPeriod = ct.matchup_end_date()

Matchup end date is 2020-02-02
Matchup period is 15


### Loop through teams

In [5]:
for teamId in range(1,13):
    print(teamId)
    results = ct.pull_matchup_data(teamId, injured_out, matchupPeriod)
    team_players_df = results['team_players']
    opponent_players_df = results['opponent_players']
    team_totals = results['team_totals']
    opponent_totals = results['opponent_totals']
    team_samples_df = ct.simulate_data(n_samples, matchup_end_date, team_players_df)
    opponent_samples_df = ct.simulate_data(n_samples, matchup_end_date, opponent_players_df)
    team_totals_samples_df = ct.generate_team_totals(n_samples, team_samples_df, team_totals)
    opponent_totals_samples_df = ct.generate_team_totals(n_samples, opponent_samples_df, opponent_totals)
    results_df = ct.compare_totals(teamId, team_totals_samples_df, opponent_totals_samples_df)

1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
# need to write some code to pull the matchup results

### Do for my team only

In [4]:
#teamId = 10
#results = ct.pull_matchup_data(teamId, injured_out, matchupPeriod)
#team_players_df = results['team_players']
#opponent_players_df = results['opponent_players']
#team_totals = results['team_totals']
#opponent_totals = results['opponent_totals']
# add season end year to boxscores and filter by that?
#team_samples_df = ct.simulate_data(n_samples, matchup_end_date, team_players_df)
#opponent_samples_df = ct.simulate_data(n_samples, matchup_end_date, opponent_players_df)
#team_totals_samples_df = ct.generate_team_totals(n_samples, team_samples_df, team_totals)
#opponent_totals_samples_df = ct.generate_team_totals(n_samples, opponent_samples_df, opponent_totals)
#results_df = ct.compare_totals(teamId, team_totals_samples_df, opponent_totals_samples_df)

### Test get n_games per day for my team

In [5]:
# this is not currently needed/used in the simulations, but might be useful when I figure out how to handle
# situations where more players have games than there are slots to start them

In [13]:
#games = ct.get_n_games(matchup_end_date, opponent_players_df)
#games.start_date.value_counts(), games.start_date.value_counts().sum()

In [14]:
#games = ct.get_n_games(matchup_end_date, team_players_df)
#games.start_date.value_counts(), games.start_date.value_counts().sum()